In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os 

In [2]:
# Initialize the master DataFrame
master_df = pd.read_csv("ZIPdf.csv")
master_df.rename(columns={"Unnamed: 0" : "ZIP"}, inplace=True)
master_df["ZIP"] = master_df["ZIP"].astype(str).str.zfill(5)
master_df.set_index("ZIP", inplace=True)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Gini,UR,%BachOrMore,STCOUNTYFP,W_Avg Emp Min Share,W_Avg Emp Black Share,W_Avg Emp White Share,W_Avg Emp Asian Share,W_Avg Emp Hisp Share,W_Avg Emp Non-Hisp Share
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['36103'],0.149040,NaN,0.859492,0.083086,0.056364,0.892998
00601,NaN,NaN,NaN,NaN,99.785408,99.779608,0.005800,99.669412,0.000000,0.000000,...,44.90,34.986667,14.562560,"['72001', '72113']",NaN,NaN,NaN,NaN,NaN,NaN
00602,NaN,NaN,NaN,NaN,99.432726,99.406094,0.026633,99.280920,0.026633,0.007990,...,49.26,11.262690,21.817471,"['72003', '72005']",NaN,NaN,NaN,NaN,NaN,NaN
00603,NaN,NaN,NaN,NaN,98.827403,98.666774,0.160630,98.377640,0.098386,0.010039,...,57.02,20.851813,24.267387,"['72005', '72071']",NaN,NaN,NaN,NaN,NaN,NaN
00606,NaN,NaN,NaN,NaN,99.480934,99.460970,0.019964,99.301258,0.039928,0.000000,...,43.73,12.011372,10.361217,"['72093', '72153']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['53063'],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['53063'],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['02020'],0.135079,0.015303,0.751177,0.072248,0.027810,0.826074


# PPP ZIP Data

In [3]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "PPP Files.csv")
file_df["Completed"] = 0
file_df

,File,Completed
0,public_150k_plus_230630.csv,0
1,public_up_to_150k_1_230630.csv,0
2,public_up_to_150k_2_230630.csv,0
3,public_up_to_150k_3_230630.csv,0
4,public_up_to_150k_4_230630.csv,0
5,public_up_to_150k_5_230630.csv,0
6,public_up_to_150k_6_230630.csv,0
7,public_up_to_150k_7_230630.csv,0
8,public_up_to_150k_8_230630.csv,0
9,public_up_to_150k_9_230630.csv,0


In [4]:
def assign_value_based_on_date(date):
    if date < pd.Timestamp('2020-04-17'):
        return '_1'
    elif date < pd.Timestamp('2020-08-09'):
        return '_2'
    else:
        return '_3'
traunches = ["_1", "_2", "_3"]

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
master_df["Number of Loans_1"] = 0
master_df["Total Loans Amount_1"] = 0
master_df["Total Employees_1"] = 0

master_df["Number of Loans_2"] = 0
master_df["Total Loans Amount_2"] = 0
master_df["Total Employees_2"] = 0

master_df["Number of Loans_3"] = 0
master_df["Total Loans Amount_3"] = 0
master_df["Total Employees_3"] = 0

In [7]:
for file in file_df[file_df["Completed"] == 0]["File"]:
    file_name = base + file
    # Check if file exists
    if not os.path.exists(file_name):
        print(f"File {file_name} does not exist. Skipping...")
        continue

    print(f"Processing {file}...")
    
    # Read file row by row since cannot fit in DF
    chunk_size = 80000
    for chunk in pd.read_csv(file_name, chunksize=chunk_size, usecols=['NonProfit', 
                                                                           'BusinessType', 
                                                                           'NAICSCode', 
                                                                           'BorrowerZip', 
                                                                           'DateApproved', 
                                                                           'InitialApprovalAmount', 
                                                                           'JobsReported']):
        # Filter chunk to only needed observations
        chunk = chunk[chunk["NonProfit"] != "Y"]
        chunk = chunk[~chunk["BusinessType"].isin(["Sole Proprietorship",
                                                   "Independent Contractors",
                                                   "Self-Employed Individuals"])]
        chunk = chunk[~chunk["NAICSCode"].isin([999990])]
        #chunk = chunk[chunk["NAICSCode"].notnull()]
        chunk.dropna(subset=["BorrowerZip"], inplace=True)

        # Edit chunk to include necessary info
        chunk["DateApproved"] = pd.to_datetime(chunk["DateApproved"])
        chunk["Traunch"] = chunk["DateApproved"].apply(assign_value_based_on_date)
        chunk["BorrowerZip"] = chunk["BorrowerZip"].astype(str).str[:5]
            
        # Aggregate Data
        grouped = chunk.groupby(['BorrowerZip', 'Traunch']).agg({
                'InitialApprovalAmount': 'sum',
                'JobsReported': 'sum',
                'Traunch': 'size'  # Count of loans
            })

        # Update master_df
        for (zip_, traunch), row in grouped.iterrows():
            if zip_ in master_df.index:
                master_df.loc[zip_, f"Number of Loans{traunch}"] += row['Traunch']  # Update the number of loans
                master_df.loc[zip_, f"Total Loans Amount{traunch}"] += row['InitialApprovalAmount']
                master_df.loc[zip_, f"Total Employees{traunch}"] += row['JobsReported'] 
            else:
                # Create a new row filled with NaNs but update the relevant columns
                new_row = pd.Series(name=zip_, dtype='float64')  # Create a Series with name set to the ZIP code
                new_row[f"Number of Loans{traunch}"] = row['Traunch']
                new_row[f"Total Loans Amount{traunch}"] = row['InitialApprovalAmount']
                new_row[f"Total Employees{traunch}"] = row['JobsReported']
        
                # Append the new row to the master DataFrame
                master_df = master_df.append(new_row)
                #print("Created new row for ZIP code: " + zip_)
                        
    file_df.loc[file_df["File"] == file, "Completed"] = 1
    
    print(f"Updated master DataFrame with data from {file}.")
    
    # Ask for user confirmation to proceed to the next file
#     user_input = input("Do you want to continue with the next file? (y/n): ")
#     if user_input.lower() != 'y':
#         print("Stopping the process.")
        # Save the master DataFrame to disk
master_df.to_csv('ZIPdf.csv')
file_df.to_csv(base + "PPP Files.csv", index=False)
        #break

Processing public_150k_plus_230630.csv...
Updated master DataFrame with data from public_150k_plus_230630.csv.
Processing public_up_to_150k_1_230630.csv...
Updated master DataFrame with data from public_up_to_150k_1_230630.csv.
Processing public_up_to_150k_2_230630.csv...
Updated master DataFrame with data from public_up_to_150k_2_230630.csv.
Processing public_up_to_150k_3_230630.csv...
Updated master DataFrame with data from public_up_to_150k_3_230630.csv.
Processing public_up_to_150k_4_230630.csv...
Updated master DataFrame with data from public_up_to_150k_4_230630.csv.
Processing public_up_to_150k_5_230630.csv...
Updated master DataFrame with data from public_up_to_150k_5_230630.csv.
Processing public_up_to_150k_6_230630.csv...
Updated master DataFrame with data from public_up_to_150k_6_230630.csv.
Processing public_up_to_150k_7_230630.csv...
Updated master DataFrame with data from public_up_to_150k_7_230630.csv.
Processing public_up_to_150k_8_230630.csv...
Updated master DataFrame 

In [8]:
master_df["Total Loans"] = master_df["Number of Loans_1"].add(master_df["Number of Loans_2"], fill_value = 0).add(master_df["Number of Loans_3"], fill_value = 0)
master_df["Total Loan Amount"] = master_df["Total Loans Amount_1"].add(master_df["Total Loans Amount_2"], fill_value = 0).add(master_df["Total Loans Amount_3"], fill_value = 0)
master_df["Total Employees"] = master_df["Total Employees_1"].add(master_df["Total Employees_2"], fill_value = 0).add(master_df["Total Employees_3"], fill_value = 0)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Total Employees_1,Number of Loans_2,Total Loans Amount_2,Total Employees_2,Number of Loans_3,Total Loans Amount_3,Total Employees_3,Total Loans,Total Loan Amount,Total Employees
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0
00601,NaN,NaN,NaN,NaN,99.785408,99.779608,0.005800,99.669412,0.000000,0.000000,...,117.0,43.0,846924.57,335.0,36.0,786849.32,341.0,81.0,1962033.89,793.0
00602,NaN,NaN,NaN,NaN,99.432726,99.406094,0.026633,99.280920,0.026633,0.007990,...,601.0,162.0,4301386.61,1381.0,119.0,3907995.21,1131.0,300.0,10408277.68,3113.0
00603,NaN,NaN,NaN,NaN,98.827403,98.666774,0.160630,98.377640,0.098386,0.010039,...,1266.0,239.0,7976491.54,2700.0,162.0,8897252.89,2606.0,423.0,21958044.43,6572.0
00606,NaN,NaN,NaN,NaN,99.480934,99.460970,0.019964,99.301258,0.039928,0.000000,...,3.0,7.0,109067.00,42.0,5.0,84750.00,28.0,13.0,198617.00,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,25015.00,2.0,NaN,NaN,NaN,1.0,25015.00,2.0
54704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,45612.00,6.0,NaN,NaN,NaN,1.0,45612.00,6.0
54860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,22200.00,3.0,NaN,NaN,NaN,1.0,22200.00,3.0


In [9]:
master_df[["Number of Loans_1",
           "Total Loans Amount_1",
           "Total Employees_1",
           "Number of Loans_2",
           "Total Loans Amount_2",
           "Total Employees_2", 
           "Number of Loans_3",
           "Total Loans Amount_3",
           "Total Employees_3",
           "Total Loans",
           "Total Loan Amount",
           "Total Employees"]] = master_df[["Number of Loans_1",
           "Total Loans Amount_1",
           "Total Employees_1",
           "Number of Loans_2",
           "Total Loans Amount_2",
           "Total Employees_2", 
           "Number of Loans_3",
           "Total Loans Amount_3",
           "Total Employees_3",
           "Total Loans",
           "Total Loan Amount",
           "Total Employees"]].fillna(0)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Total Employees_1,Number of Loans_2,Total Loans Amount_2,Total Employees_2,Number of Loans_3,Total Loans Amount_3,Total Employees_3,Total Loans,Total Loan Amount,Total Employees
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0
00601,NaN,NaN,NaN,NaN,99.785408,99.779608,0.005800,99.669412,0.000000,0.000000,...,117.0,43.0,846924.57,335.0,36.0,786849.32,341.0,81.0,1962033.89,793.0
00602,NaN,NaN,NaN,NaN,99.432726,99.406094,0.026633,99.280920,0.026633,0.007990,...,601.0,162.0,4301386.61,1381.0,119.0,3907995.21,1131.0,300.0,10408277.68,3113.0
00603,NaN,NaN,NaN,NaN,98.827403,98.666774,0.160630,98.377640,0.098386,0.010039,...,1266.0,239.0,7976491.54,2700.0,162.0,8897252.89,2606.0,423.0,21958044.43,6572.0
00606,NaN,NaN,NaN,NaN,99.480934,99.460970,0.019964,99.301258,0.039928,0.000000,...,3.0,7.0,109067.00,42.0,5.0,84750.00,28.0,13.0,198617.00,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,25015.00,2.0,0.0,0.00,0.0,1.0,25015.00,2.0
54704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,45612.00,6.0,0.0,0.00,0.0,1.0,45612.00,6.0
54860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,22200.00,3.0,0.0,0.00,0.0,1.0,22200.00,3.0


In [10]:
master_df["Loans per Emp Bus"] = master_df["Total Loans"] / master_df["Total Emp Bus"]
master_df["Avg Loan Amt per Emp"] = master_df["Total Loan Amount"] / master_df["CBP Emp"]
master_df["Loans per Emp Bus_1"] = master_df["Number of Loans_1"] / master_df["Total Emp Bus"]
master_df["Avg Loan Amt per Emp_1"] = master_df["Total Loans Amount_1"] / master_df["CBP Emp"]
master_df["Loans per Emp Bus_2"] = master_df["Number of Loans_2"] / master_df["Total Emp Bus"]
master_df["Avg Loan Amt per Emp_2"] = master_df["Total Loans Amount_2"] / master_df["CBP Emp"]
master_df["Loans per Emp Bus_3"] = master_df["Number of Loans_3"] / master_df["Total Emp Bus"]
master_df["Avg Loan Amt per Emp_3"] = master_df["Total Loans Amount_3"] / master_df["CBP Emp"]
master_df.replace([np.inf, -np.inf], np.nan, inplace=True)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Total Loan Amount,Total Employees,Loans per Emp Bus,Avg Loan Amt per Emp,Loans per Emp Bus_1,Avg Loan Amt per Emp_1,Loans per Emp Bus_2,Avg Loan Amt per Emp_2,Loans per Emp Bus_3,Avg Loan Amt per Emp_3
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00601,NaN,NaN,NaN,NaN,99.785408,99.779608,0.005800,99.669412,0.000000,0.000000,...,1962033.89,793.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00602,NaN,NaN,NaN,NaN,99.432726,99.406094,0.026633,99.280920,0.026633,0.007990,...,10408277.68,3113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00603,NaN,NaN,NaN,NaN,98.827403,98.666774,0.160630,98.377640,0.098386,0.010039,...,21958044.43,6572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00606,NaN,NaN,NaN,NaN,99.480934,99.460970,0.019964,99.301258,0.039928,0.000000,...,198617.00,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25015.00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45612.00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22200.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EIDL Advance ZIP Data

In [11]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "EIDL Advance Files.csv")
file_df

,File,Completed
0,01 EIDL Advance through 111520.csv,1
1,02 EIDL Advance through 111520.csv,1
2,03 EIDL Advance through 111520.csv,1
3,04 EIDL Advance through 111520.csv,1
4,05 EIDL Advance through 111520.csv,1
5,06 EIDL Advance through 111520.csv,1
6,07 EIDL Advance through 111520.csv,1


In [12]:
# master_df[["Number of Advs EIDLA", "Total Advs Amount EIDLA"]] = 0

In [13]:
# for file in file_df[file_df["Completed"] == 0]["File"]:
#     file_name = base + file
#     # Check if file exists
#     if not os.path.exists(file_name):
#         print(f"File {file_name} does not exist. Skipping...")
#         continue

#     print(f"Processing {file}...")

#     chunk_size = 150000

#     for chunk in pd.read_csv(file_name, chunksize=chunk_size, usecols=['LEGALENTITYZIP5', 
#                                                                        'FEDERALACTIONOBLIGATION',
#                                                                        'ACTIONTYPE']):
#         chunk.dropna(subset=["LEGALENTITYZIP5"], inplace=True)
    
#         # Edit chunk to include necessary info
#         chunk["LEGALENTITYZIP5"] = chunk["LEGALENTITYZIP5"].astype(str).str.zfill(5)
#             # The data for advances contains instances where funds were returned by banks or rescinded by the 
#             # government after too much was awarded. These negative values are important to the sum of funding 
#             # recieved, but need to be accounted for when obtaining counts of advances 
#         chunk['Adv_Count'] = chunk['ACTIONTYPE'].apply(lambda x: 1 if x == "A" else 0)
    
#         # Aggregate Data
#         grouped = chunk.groupby(['LEGALENTITYZIP5']).agg({
#             'FEDERALACTIONOBLIGATION': 'sum',
#             'Adv_Count': 'sum'  # Sum of Adv_Count will give the adjusted count of loans
#         })
    
#         # Update master_df
#         for zip_, row in grouped.iterrows():
#             if zip_ in master_df.index:
#                 master_df.loc[zip_, "Number of Advs EIDLA"] += row['Adv_Count']  # Update the number of loans
#                 master_df.loc[zip_, "Total Advs Amount EIDLA"] += row['FEDERALACTIONOBLIGATION']
#             else:
#                 # Create a new row filled with NaNs but update the relevant columns
#                 new_row = pd.Series(name=zip_, dtype='float64')  # Create a Series with name set to the ZIP code
#                 new_row["Number of Advs EIDLA"] = row['Adv_Count']
#                 new_row["Total Advs Amount EIDLA"] = row['FEDERALACTIONOBLIGATION']
        
#                 # Append the new row to the master DataFrame
#                 master_df = master_df.append(new_row)
                        
#     file_df.loc[file_df["File"] == file, "Completed"] = 1
    
#     print(f"Updated master DataFrame with data from {file}.")
    
# master_df.to_csv('ZIPdf.csv')
# file_df.to_csv(base + "EIDL Advance Files.csv", index=False)

# EIDL Loan ZIP Data

In [14]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "EIDL Loan Files.csv")
file_df

,File,Completed
0,DATAACT_EIDL_LOANS_20200401-20200609.csv,1
1,DATAACT_EIDL_LOANS_20200610-20200625.csv,1
2,DATAACT_EIDL_LOANS_20200626-20200723.csv,1
3,DATAACT_EIDL_LOANS_20200724-20201115.csv,1
4,DATAACT_EIDL_LOANS_DMCS2.0.csv,1


In [15]:
# master_df[["Number of Loans EIDLL", "Total Loan Amount EIDLL"]] = 0

In [16]:
# for file in file_df[file_df["Completed"] == 0]["File"]:
#     file_name = base + file
#     # Check if file exists
#     if not os.path.exists(file_name):
#         print(f"File {file_name} does not exist. Skipping...")
#         continue

#     print(f"Processing {file}...")
    
#     # Read file row by row since cannot fit in DF
#     chunk_size = 160000
#     for chunk in pd.read_csv(file_name, chunksize=chunk_size, usecols=['LEGALENTITYZIP5', 
#                                                                        'FACEVALUEOFDIRECTLOANORLOANGUARANTEE', 
#                                                                        'FAIN',
#                                                                        'ACTIONTYPE']):
#         chunk.dropna(subset=["LEGALENTITYZIP5"], inplace=True)

#         # Edit chunk to include necessary info
#         chunk["LEGALENTITYZIP5"] = chunk["LEGALENTITYZIP5"].astype(str).str.zfill(5)
#         chunk['Loan_Count'] = chunk['ACTIONTYPE'].apply(lambda x: 1 if x == "A" else 0)
        
#         # Aggregate Data
#         grouped = chunk.groupby(['LEGALENTITYZIP5']).agg({
#                 'FACEVALUEOFDIRECTLOANORLOANGUARANTEE': 'sum',
#                 'Loan_Count': 'size'  # Count of loans
#             })

#         # Update master_df
#         for zip_, row in grouped.iterrows():
#             if zip_ in master_df.index:
#                 master_df.loc[zip_, "Number of Loans EIDLL"] += row['Loan_Count']  # Update the number of loans
#                 master_df.loc[zip_, "Total Loan Amount EIDLL"] += row['FACEVALUEOFDIRECTLOANORLOANGUARANTEE']
#             else:
#                 # Create a new row filled with NaNs but update the relevant columns
#                 new_row = pd.Series(name=zip_, dtype='float64')  # Create a Series with name set to the ZIP code
#                 new_row["Number of Loans EIDLL"] = row['Loan_Count']
#                 new_row["Total Loan Amount EIDLL"] = row['FACEVALUEOFDIRECTLOANORLOANGUARANTEE']
        
#                 # Append the new row to the master DataFrame
#                 master_df = master_df.append(new_row)
#                 #print("Created new row for ZIP code: " + zip_)
                        
#     file_df.loc[file_df["File"] == file, "Completed"] = 1
    
#     print(f"Updated master DataFrame with data from {file}.")
    
#     # Ask for user confirmation to proceed to the next file
# #     user_input = input("Do you want to continue with the next file? (y/n): ")
# #     if user_input.lower() != 'y':
# #         print("Stopping the process.")
#         # Save the master DataFrame to disk
# master_df.to_csv('ZIPdf.csv')
# file_df.to_csv(base + "EIDL Loan Files.csv", index=False)
#         #break

# EIDL Post Nov-15 2020

In [17]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "EIDL Post Nov 20.csv")
file_df

,File,Completed
0,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
1,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
2,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
3,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
4,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
5,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
6,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
7,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
8,All_Assistance_PrimeTransactions_2023-09-09_H2...,0
9,All_Assistance_PrimeTransactions_2023-09-09_H2...,0


In [18]:
# for file in file_df[file_df["Completed"] == 0]["File"]:
#     file_name = base + file
#     # Check if file exists
#     if not os.path.exists(file_name):
#         print(f"File {file_name} does not exist. Skipping...")
#         continue

#     print(f"Processing {file}...")
    
#     # Read file row by row since cannot fit in DF
#     chunk_size = 160000
#     for chunk in pd.read_csv(file_name, chunksize=chunk_size, usecols=['cfda_number', 
#                                                                       'action_type_code', 
#                                                                       'federal_action_obligation',
#                                                                       'face_value_of_loan',
#                                                                       'disaster_emergency_fund_codes_for_overall_award',
#                                                                       'recipient_zip_code']):
#         # Filter chunk to only needed observations
#         chunk["disaster_emergency_fund_codes_for_overall_award"] = chunk[
#             "disaster_emergency_fund_codes_for_overall_award"].astype(str).str[0]
#         chunk = chunk[chunk["disaster_emergency_fund_codes_for_overall_award"].isin(["L","M","N","O","P","U","V"])]
#         chunk.dropna(subset=["recipient_zip_code"], inplace=True)
#         chunk["recipient_zip_code"] = chunk["recipient_zip_code"].astype(str).str.zfill(5)
#         chunk["Count"] = chunk['action_type_code'].apply(lambda x: 1 if x == "A" else 0)
    
#         chunk_l = chunk[chunk["cfda_number"].astype(np.double) == 59.072]
#         chunk_a = chunk[chunk["cfda_number"].astype(np.double) == 59.008]

        
#         # Aggregate Data
#         grouped_l = chunk.groupby(['recipient_zip_code']).agg({
#                 'face_value_of_loan': 'sum',
#                 'Count': 'sum'  # Count of loans
#             })
#         grouped_a = chunk.groupby(['recipient_zip_code']).agg({
#                 'federal_action_obligation': 'sum',
#                 'Count': 'sum'  # Count of advs
#             })
        
#         combined_df = pd.merge(grouped_l, grouped_a, on='recipient_zip_code', how='outer', suffixes=('_loan', '_adv'))
#         combined_df.fillna(0, inplace=True)
        
#         # Update master_df
#         for zip_, row in combined_df.iterrows():
#             if zip_ in master_df.index:
#                 master_df.loc[zip_, "Number of Loans EIDLL"] += row['Count_loan']  # Update the number of loans
#                 master_df.loc[zip_, "Total Loan Amount EIDLL"] += row['face_value_of_loan']
#                 master_df.loc[zip_, "Number of Advs EIDLA"] += row['Count_adv']  # Update the number of advs
#                 master_df.loc[zip_, "Total Advs Amount EIDLA"] += row['federal_action_obligation']
#             else:
#                 # Create a new row filled with NaNs but update the relevant columns
#                 new_row = pd.Series(name=zip_, dtype='float64')  # Create a Series with name set to the ZIP code
#                 new_row["Number of Loans EIDLL"] = row['Count_loan']
#                 new_row["Total Loan Amount EIDLL"] = row['face_value_of_loan']
#                 new_row["Number of Advs EIDLA"] = row['Count_adv']
#                 new_row["Total Advs Amount EIDLA"] = row['federal_action_obligation']
        
#                 # Append the new row to the master DataFrame
#                 master_df = master_df.append(new_row)
#                 #print("Created new row for ZIP code: " + zip_)
                        
#     file_df[file_df["File"] == file]["Completed"] = 1
    
#     print(f"Updated master DataFrame with data from {file}.")

# master_df.to_csv('ZIPdf.csv')
# file_df.to_csv(base + "EIDL Post Nov 20.csv", index=False)

In [19]:
# master_df[["Number of Advs EIDLA",
#            "Total Advs Amount EIDLA"]] = master_df[["Number of Advs EIDLA",
#                                             "Total Advs Amount EIDLA"]].fillna(0)

In [20]:
# master_df[["Number of Loans EIDLL",
#            "Total Loan Amount EIDLL"]] = master_df[["Number of Loans EIDLL",
#                                             "Total Loan Amount EIDLL"]].fillna(0)

In [21]:
# master_df["EIDL Loans per Emp Bus"] = master_df["Number of Loans EIDLL"] / master_df["Total Emp Bus"]
# master_df["Avg EIDL Loan Amt per Emp"] = master_df["Total Loan Amount EIDLL"] / master_df["CBP Emp"]
# master_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# master_df

In [22]:
# master_df["Advs per Emp Bus"] = master_df["Number of Advs EIDLA"] / master_df["Total Emp Bus"]
# master_df["Avg Adv Amt per Emp"] = master_df["Total Advs Amount EIDLA"] / master_df["CBP Emp"]
# master_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# master_df

# Stats

In [23]:
master_df.describe()

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Total Loan Amount,Total Employees,Loans per Emp Bus,Avg Loan Amt per Emp,Loans per Emp Bus_1,Avg Loan Amt per Emp_1,Loans per Emp Bus_2,Avg Loan Amt per Emp_2,Loans per Emp Bus_3,Avg Loan Amt per Emp_3
count,32109.000000,35052.000000,35052.000000,37322.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,...,4.029900e+04,40299.000000,37322.000000,3.504900e+04,37322.000000,35049.000000,37322.000000,3.504900e+04,37322.000000,35049.000000
mean,61.825968,227.137111,3638.925140,266.512513,26.309044,18.998565,7.310479,10.415032,2.316485,1.774790,...,1.635676e+07,1863.051986,0.418266,7.540021e+03,0.110698,3341.812484,0.156128,1.897559e+03,0.151440,2300.649422
std,88.443966,406.851597,8085.098199,414.475046,24.866075,20.157029,14.854133,16.399289,5.762396,9.231636,...,4.122567e+07,4252.682125,0.321156,1.646075e+04,0.119044,10303.522965,0.155452,8.288236e+03,0.140103,5336.620501
min,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,9.000000,11.000000,80.000000,24.000000,7.695221,6.483358,0.264276,1.863354,0.150038,0.082109,...,1.456726e+05,21.000000,0.183116,3.031873e+03,0.034483,788.664683,0.048387,4.038469e+02,0.050000,576.960525
50%,31.000000,41.000000,423.000000,90.000000,16.288660,10.917816,1.001669,4.099142,0.508504,0.220264,...,1.325290e+06,175.000000,0.361111,5.314183e+03,0.090909,2035.714286,0.117139,1.023640e+03,0.126214,1421.542625
75%,78.000000,255.000000,3367.000000,326.000000,37.407054,23.085094,6.340451,10.763758,1.719629,0.519481,...,1.198637e+07,1517.000000,0.618755,8.413865e+03,0.159143,3621.839080,0.230987,2.047714e+03,0.227273,2574.140499
max,1634.000000,6893.000000,177226.000000,6897.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,1.546740e+09,133008.000000,8.333333,1.592578e+06,6.000000,816564.500000,5.000000,1.168476e+06,3.000000,424102.500000


In [24]:
master_df[~master_df["Min Share"].isnull()].describe()

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Total Loan Amount,Total Employees,Loans per Emp Bus,Avg Loan Amt per Emp,Loans per Emp Bus_1,Avg Loan Amt per Emp_1,Loans per Emp Bus_2,Avg Loan Amt per Emp_2,Loans per Emp Bus_3,Avg Loan Amt per Emp_3
count,29574.000000,30771.000000,30771.000000,32638.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,...,3.363100e+04,33631.000000,32638.000000,30771.000000,32638.000000,30771.000000,32638.000000,30771.000000,32638.000000,30771.000000
mean,66.360283,256.658022,4069.889571,302.106839,26.309044,18.998565,7.310479,10.415032,2.316485,1.774790,...,1.944207e+07,2215.454640,0.432061,7309.194868,0.109256,3056.306189,0.156429,1748.918154,0.166376,2503.970525
std,90.643775,425.862046,8515.727201,431.575897,24.866075,20.157029,14.854133,16.399289,5.762396,9.231636,...,4.445869e+07,4569.227086,0.304393,10019.553411,0.098997,5480.280641,0.142527,3540.195425,0.135005,4765.052984
min,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,15.000000,95.000000,40.000000,7.695221,6.483358,0.264276,1.863354,0.150038,0.082109,...,3.066152e+05,45.000000,0.200000,3452.051810,0.041667,1026.714752,0.054054,491.124612,0.072464,867.355632
50%,36.000000,59.000000,572.000000,119.000000,16.288660,10.917816,1.001669,4.099142,0.508504,0.220264,...,2.358688e+06,309.000000,0.381164,5532.456152,0.093878,2147.066663,0.120000,1092.791489,0.142857,1593.512000
75%,84.000000,320.000000,4217.500000,394.000000,37.407054,23.085094,6.340451,10.763758,1.719629,0.519481,...,1.797554e+07,2251.000000,0.634921,8482.814373,0.157895,3645.794648,0.232393,2066.431891,0.239337,2757.000028
max,1634.000000,6893.000000,177226.000000,6897.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,1.546740e+09,133008.000000,6.000000,336934.333333,3.000000,331865.666667,4.000000,229710.835000,2.090909,225840.500000


In [25]:
stats_df = master_df[~master_df["Min Share"].isnull()][["Min Share",
                                                       "Min Share Excl B",
                                                       "Black Share",
                                                       "White Share",
                                                       "Asian Share",
                                                       "Hisp Share",
                                                       "Native Share",
                                                       'Rural', 
                                                       'PC Inc', 
                                                       'Gini',
                                                       'UR', 
                                                       '%BachOrMore',
                                                       "W_Avg Emp Min Share",
                                                       "W_Avg Emp Black Share",
                                                       "W_Avg Emp White Share",
                                                       "W_Avg Emp Asian Share",
                                                       "W_Avg Emp Hisp Share",
                                                       "W_Avg Emp Non-Hisp Share",
                                                       "W_Avg Emp Min Share",
                                                       "W_Avg Emp Min Share",
                                                       "Total Pop",
                                                       "Loans per Emp Bus",
                                                       "Avg Loan Amt per Emp",
                                                       "Loans per Emp Bus_1",
                                                       "Avg Loan Amt per Emp_1",
                                                       "Loans per Emp Bus_2",
                                                       "Avg Loan Amt per Emp_2",
                                                       "Loans per Emp Bus_3",
                                                       "Avg Loan Amt per Emp_3"]]
stats_df.describe()

,Min Share,Min Share Excl B,Black Share,White Share,Asian Share,Hisp Share,Native Share,Rural,PC Inc,Gini,...,W_Avg Emp Min Share,Total Pop,Loans per Emp Bus,Avg Loan Amt per Emp,Loans per Emp Bus_1,Avg Loan Amt per Emp_1,Loans per Emp Bus_2,Avg Loan Amt per Emp_2,Loans per Emp Bus_3,Avg Loan Amt per Emp_3
count,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,32763.000000,32405.000000,...,15979.000000,33631.000000,32638.000000,30771.000000,32638.000000,30771.000000,32638.000000,30771.000000,32638.000000,30771.000000
mean,26.309044,18.998565,7.310479,73.690956,2.316485,10.415032,1.774790,64.457200,34022.115740,41.173309,...,0.143364,9952.918022,0.432061,7309.194868,0.109256,3056.306189,0.156429,1748.918154,0.166376,2503.970525
std,24.866075,20.157029,14.854133,24.866075,5.762396,16.399289,9.231636,44.097957,16923.415799,8.190941,...,0.103154,14935.801329,0.304393,10019.553411,0.098997,5480.280641,0.142527,3540.195425,0.135005,4765.052984
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,289.000000,0.020000,...,0.013952,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.695221,6.483358,0.264276,62.592946,0.150038,1.863354,0.082109,8.526885,24841.000000,37.350000,...,0.071605,664.500000,0.200000,3452.051810,0.041667,1026.714752,0.054054,491.124612,0.072464,867.355632
50%,16.288660,10.917816,1.001669,83.711340,0.508504,4.099142,0.220264,100.000000,30756.000000,41.570000,...,0.110560,2681.000000,0.381164,5532.456152,0.093878,2147.066663,0.120000,1092.791489,0.142857,1593.512000
75%,37.407054,23.085094,6.340451,92.304779,1.719629,10.763758,0.519481,100.000000,38835.000000,45.770000,...,0.187418,13459.000000,0.634921,8482.814373,0.157895,3645.794648,0.232393,2066.431891,0.239337,2757.000028
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,701688.000000,82.760000,...,0.679245,135256.000000,6.000000,336934.333333,3.000000,331865.666667,4.000000,229710.835000,2.090909,225840.500000


In [26]:
excluded_prefixes = ('962',
'963',
'964',
'965',
'966',
'090',
'091',
'092',
'093',
'094',
'095',
'096',
'097',
'098',
'006',
'007',
'008',
'009')

stats_df = stats_df[~stats_df.index.str.startswith(excluded_prefixes)]
stats_df

,Min Share,Min Share Excl B,Black Share,White Share,Asian Share,Hisp Share,Native Share,Rural,PC Inc,Gini,...,W_Avg Emp Min Share,Total Pop,Loans per Emp Bus,Avg Loan Amt per Emp,Loans per Emp Bus_1,Avg Loan Amt per Emp_1,Loans per Emp Bus_2,Avg Loan Amt per Emp_2,Loans per Emp Bus_3,Avg Loan Amt per Emp_3
ZIP,,,,,,,,,,,,,,,,,,,,,
01001,15.390956,13.312529,2.078427,84.609044,2.890956,6.771079,0.135422,0.965615,41144.0,42.84,...,0.104759,16984.0,0.731250,5856.146415,0.245833,3352.367207,0.245833,902.597423,0.239583,1601.181785
01002,36.718920,31.678641,5.040279,63.281080,15.712316,9.902751,0.166921,14.972059,38627.0,54.44,...,NaN,27558.0,0.569468,4456.181504,0.145798,1715.197631,0.202401,1359.536721,0.221269,1381.447152
01003,39.568400,32.943484,6.624915,60.431600,20.176564,8.737644,0.105636,0.000000,4935.0,50.23,...,0.069946,13253.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
01005,9.081633,8.142857,0.938776,90.918367,0.612245,3.122449,0.142857,100.000000,39783.0,33.77,...,0.094359,4900.0,0.364780,4303.253113,0.132075,2280.721763,0.106918,704.397612,0.125786,1318.133737
01007,13.311288,11.567140,1.744148,86.688712,2.379563,4.279323,0.038903,71.970434,43450.0,38.15,...,0.069946,15423.0,0.534202,6553.248685,0.175896,2655.974900,0.172638,1852.649565,0.185668,2044.624220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99923,10.606061,10.606061,0.000000,89.393939,0.000000,3.030303,0.000000,100.000000,NaN,NaN,...,NaN,66.0,0.250000,496.363636,0.000000,0.000000,0.250000,496.363636,0.000000,0.000000
99925,62.627551,62.117347,0.510204,37.372449,1.020408,1.785714,44.005102,100.000000,30746.0,44.80,...,NaN,784.0,0.333333,5580.136255,0.066667,1121.355378,0.100000,1836.290837,0.166667,2622.490040
99926,90.332907,90.076825,0.256082,9.667093,0.000000,2.944942,74.327785,100.000000,24979.0,38.18,...,NaN,1562.0,0.083333,895.882353,0.000000,0.000000,0.083333,895.882353,0.000000,0.000000


In [27]:
def weighted_average(df, values_col, weights_col):
    # Element-wise multiplication, ignoring NaNs
    weighted_values = df[values_col] * df[weights_col]
    
    # Sum of weighted values and weights, ignoring NaNs
    sum_weighted_values = weighted_values.sum(skipna=True)
    sum_weights = df[weights_col].sum(skipna=True)
    
    # Calculate weighted average, returning NaN if not computable
    if sum_weights == 0:
        return np.nan
    else:
        return sum_weighted_values / sum_weights

In [29]:
for column in stats_df.columns:
    if column != "Total Pop":
        print(column)
        sub_data = master_df[~master_df["Min Share"].isnull()][[column, "Total Pop"]]
        print(weighted_average(sub_data, column, "Total Pop"))

Min Share
42.723896750764816
Min Share Excl B
30.79049627686281
Black Share
11.933400473902005
White Share
57.276103249235184
Asian Share
5.861855861069847
Hisp Share
19.51655641718283
Native Share
0.6724129764822445
Rural
19.88455407593475
PC Inc
37512.35111359813
Gini
43.58688009048675
UR
5.499028274607905
%BachOrMore
33.40044956532447
W_Avg Emp Min Share
0.1442346015537462
W_Avg Emp Black Share
0.014944787172054962
W_Avg Emp White Share
0.7328978332593434
W_Avg Emp Asian Share
0.07553803104477251
W_Avg Emp Hisp Share
0.04269534723950568
W_Avg Emp Non-Hisp Share
0.7848242392247963
W_Avg Emp Min Share
0.1442346015537462
W_Avg Emp Min Share
0.1442346015537462
Loans per Emp Bus
0.6275647208920458
Avg Loan Amt per Emp
5615.9467600594535
Loans per Emp Bus_1
0.13324961447963563
Avg Loan Amt per Emp_1
2298.525102694748
Loans per Emp Bus_2
0.25028823998919236
Avg Loan Amt per Emp_2
1550.4680565464723
Loans per Emp Bus_3
0.24402686642321786
Avg Loan Amt per Emp_3
1766.9536008182338


In [30]:
stats_df.to_csv("Calculations.csv")

In [31]:
master_df.to_csv("ZIPdf.csv")